# Text Classification

In [62]:
#Imports

import pandas as pd
import numpy as np
import re
from string import punctuation, whitespace
import nltk
from nltk.corpus import stopwords
from pattern.it import parse, split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [63]:
#Load clinical diaries
df =pd.read_excel('data/DATI PS 2020-CON NOTE DIARIO CLINICO.xlsx')#,usecols="A:C,E:F")
df = df.rename(columns={"MEDICO" : "AUTORE"})

df =df.append(pd.read_excel('data/DATI PS 2019-CON NOTE DIARIO CLINICO.xlsx'), ignore_index=True)

df.columns = map(str.lower, df.columns)

df.drop('autore', inplace=True, axis=1)

In [64]:
df.head()

,numero_pratica,tipo_nota,data_inserimento,testo
0,PS2019071726,Nota clinica,"04/12/2019 12:21:50,759000",si esegue rx endorale che evidenzia la presenz...
1,PS2020000210,Nota clinica,"02/01/2020 09:52:44,668000",Previo consenso informato e previa anestesia p...
2,PS2020000211,Nota clinica,"02/01/2020 09:30:43,247000","presa visione della opt portata dal pz, previo..."
3,PS2020000214,Nota clinica,"02/01/2020 11:00:29,157000",Si esegue rx endorale che conferma presenza di...
4,PS2020000216,Nota clinica,"02/01/2020 09:31:38,871000",In accordo con il pz si procede alla ricementa...


In [65]:
df.shape

(8383, 4)

In [66]:
df.dtypes

numero_pratica      object
tipo_nota           object
data_inserimento    object
testo               object
dtype: object

In [67]:
#change datatypes
df['testo']= df['testo'].astype(str)
df['numero_pratica']= df['numero_pratica'].astype(str)

In [68]:
#
df = df.groupby(['numero_pratica'], as_index = False).agg({'testo': ' '.join})

df = df[df['numero_pratica'] !='NUMERO_PRATICA']

In [69]:
df['numero_pratica'].value_counts()

PS2019048324    1
PS2020016595    1
PS2019002116    1
PS2020013163    1
PS2019015530    1
               ..
PS2019049917    1
PS2019026808    1
PS2020017557    1
PS2019026325    1
PS2020012325    1
Name: numero_pratica, Length: 8163, dtype: int64

## 1. Text cleaning and preparation

### 1.1. Special character and punctuation signs cleaning

In [70]:
#Special characters (more than in punctuation list)
df['testo_clean'] = df['testo'].apply(lambda x: re.sub(r'[^\w\s]', ' ', x) )

In [71]:
#Whitespaces cleaning

whites = list(set(whitespace)-{' '})
for w in whites:
    df['testo_clean']= df['testo_clean'].str.replace(w,' ')

### 1.2. Upcase/downcase

In [72]:
df['testo_clean'] = df['testo_clean'].str.lower()

### 1.3. Stop words

In [73]:
#Import stopwords
stop = stopwords.words('italian')

for s in stop:
    df['testo_clean']= df['testo_clean'].str.replace(r"\b" + s + r"\b",' ')

c:\program files\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


### 1.4. Remove double spaces

In [74]:
#Remove double spaces
df['testo_clean'] = df['testo_clean'].apply(lambda x: re.sub(' +', ' ', x) )

### 1.5. Lemmatization

I chose to apply lemmatization only as I did not want to produce words that do not exist.

In [75]:
#lemmatize word function
def lemmatize_word(input_word):
    in_word = input_word#.decode('utf-8')
    # print('Something: {}'.format(in_word))
    word_it = parse(
        in_word,
        tokenize=False,
        tag=False,
        chunk=False,
        lemmata=True
    )
    # print("Input: {} Output: {}".format(in_word, word_it))
    the_lemmatized_word = word_it.split()[0][0][4]
    # print("Returning: {}".format(the_lemmatized_word))
    return the_lemmatized_word

#tokenize sentence (string) 
def tokenize(sentence_totoken):
    return nltk.tokenize.word_tokenize(sentence_totoken)
    
#tokenize and lemmatize sentences and return string
def lemmatize_sentence(sentence):
    lemmatized = []
    for word in tokenize(sentence):
        lemmatized.append(lemmatize_word(word))
    lemmatized_text = " ".join(lemmatized)
    return lemmatized_text
    

In [76]:
df['testo_clean'] = df['testo_clean'].apply(lambda x : lemmatize_sentence(x) )

In [77]:
df.head()

,numero_pratica,testo,testo_clean
1,PS2019000196,Si esegue Rx endorale da cui si evidenzia cari...,eseguire rx endorala evidenziare carie mesiala...
2,PS2019000197,si esegue rx endorale di controllo. Si spiega ...,eseguire rx endorala controllo spiegare pazien...
3,PS2019000198,eseguita rx endorale che conferma quadro clini...,eseguitare rx endorala conferma quadro clinico...
4,PS2019000199,Eseguita rx endorale che evidenzia estensione ...,eseguitare rx endorala evidenziare estensione ...
5,PS2019000200,Si eseguono rx endorali che non evidenziano ri...,eseguire rx endorale evidenziare rima frattura...


In [78]:
#Load categorized and uncategorized 
df_classified =pd.read_excel('data/Odontoiatria PS 2019-2020_21-02_05-05.xlsx',sheet_name= 'Data Base')
df_classified.columns = map(str.lower, df_classified.columns)

df_classified.head()

db_originale=pd.read_excel('data/Odontoiatria PS 2019-2020_21-02_05-05.xlsx',sheet_name= 'Data Base')
db_originale.columns = map(str.lower, db_originale.columns)

In [79]:
df_classified.shape

(3416, 21)

In [80]:
#Select only relevant columns
df_classified= df_classified[['anno', 'mese', 'data_accettazione', 'modalita_dimissione', 'numero_sdo_ricetta', 'eta_pz', 'adulti-bambini', 'data_ora_ingresso_ps', 'data_ora_uscita_ps', 'diagnosi_principale', 'diagnosi_1', 'pz_raggruppamento_residenza']]

In [81]:
#Remove duplicates and reindex
df_classified = df_classified.drop_duplicates().reset_index(drop=True)

In [82]:
#Load aggregated diagnosis
diagnosi_aggr=pd.read_excel('data/diagnosi_aggr.xlsx')
diagnosi_aggr.columns = map(str.lower, diagnosi_aggr.columns)

diagnosi_aggr.head()

,diagnosi_principale,motivo_accesso
0,V722 - VISITA ODONTOIATRICA,VISITA
1,V6759 - ALTRA VISITA DI CONTROLLO,VISITA
2,V523 - COLLOCAZIONE E SISTEMAZIONE DI PROTESI ...,ALTRO
3,5206 - DISTURBI DELLERUZIONE DEL DENTE,ALTRO
4,"52879 - ALTRI DISTURBI DELLEPITELIO ORALE, INC...",ALTRO


In [83]:
#Join full database with aggregated diagnosis

df_classified = df_classified.merge(diagnosi_aggr,how='left',left_on='diagnosi_principale', right_on='diagnosi_principale')
df_classified.shape

(2252, 13)

In [84]:
#Join full database with clinical diary

df_classified = df_classified.merge(df,how='left',left_on='numero_sdo_ricetta', right_on='numero_pratica')


In [85]:
df_classified.head()

,anno,mese,data_accettazione,modalita_dimissione,numero_sdo_ricetta,eta_pz,adulti-bambini,data_ora_ingresso_ps,data_ora_uscita_ps,diagnosi_principale,diagnosi_1,pz_raggruppamento_residenza,motivo_accesso,numero_pratica,testo,testo_clean
0,2019,2,2019-02-21,1 - Dimissione ordinaria al domicilio del pazi...,PS2019011263,24,ADULTO,2019-02-21 08:52:58,2019-02-21 10:00:00,5253 - RADICE DENTARIA RITENUTA,5253 - RADICE DENTARIA RITENUTA,1 - IN ASL,ALTRO,PS2019011263,pz in ps per algia I quadrante.All'eo evidenzi...,pz ps algia quadrante eo evidenziare residuo r...
1,2019,2,2019-02-21,1 - Dimissione ordinaria al domicilio del pazi...,PS2019011267,44,ADULTO,2019-02-21 08:59:37,2019-02-21 09:23:00,V722 - VISITA ODONTOIATRICA,V722 - VISITA ODONTOIATRICA,1 - IN ASL,VISITA,PS2019011267,Consultata opt eseguita durante accesso in ps ...,consultatare opt eseguitare durante accesso ps...
2,2019,2,2019-02-21,1 - Dimissione ordinaria al domicilio del pazi...,PS2019011268,28,ADULTO,2019-02-21 09:02:33,2019-02-21 10:02:00,52102 - CARIE DENTALE ESTESA ALLA DENTINA,52102 - CARIE DENTALE ESTESA ALLA DENTINA,1 - IN ASL,CARIE,PS2019011268,eseguita OPT dalla quale si evidenzia vicinanz...,eseguitare opt evidenziare vicinanza nare rima...
3,2019,2,2019-02-21,1 - Dimissione ordinaria al domicilio del pazi...,PS2019011281,32,ADULTO,2019-02-21 09:37:42,2019-02-21 11:17:00,5253 - RADICE DENTARIA RITENUTA,5253 - RADICE DENTARIA RITENUTA,2 - IN REGIONE,ALTRO,PS2019011281,Eseguita rx endorale che conferma diagnosi cli...,eseguitare rx endorala conferma diagnosi clini...
4,2019,2,2019-02-21,1 - Dimissione ordinaria al domicilio del pazi...,PS2019011283,66,ADULTO,2019-02-21 09:53:29,2019-02-21 10:44:00,52109 - CARIE DENTALE,52109 - CARIE DENTALE,1 - IN ASL,CARIE,PS2019011283,eseguita rx endorale II q. si evidenzia lesion...,eseguitare rx endorala ii q evidenziare lesion...


In [86]:
df_classified.dtypes

anno                                    int64
mese                                    int64
data_accettazione              datetime64[ns]
modalita_dimissione                    object
numero_sdo_ricetta                     object
eta_pz                                  int64
adulti-bambini                         object
data_ora_ingresso_ps           datetime64[ns]
data_ora_uscita_ps             datetime64[ns]
diagnosi_principale                    object
diagnosi_1                             object
pz_raggruppamento_residenza            object
motivo_accesso                         object
numero_pratica                         object
testo                                  object
testo_clean                            object
dtype: object

In [87]:
df_classified.shape

(2252, 16)

In [88]:
df_classified['diagnosi_principale'].value_counts()

V722 - VISITA ODONTOIATRICA                                                              501
52100 - CARIE DENTALE NON SPECIFICATA                                                    249
5226 - PERIODONTITE CRONICA APICALE                                                      184
52109 - CARIE DENTALE                                                                    157
68100 - FLEMMONE E ASCESSO,NON SPECIFICATO                                               156
                                                                                        ... 
72885 - CONTRATTURA MUSCOLARE                                                              1
8024 - FRATTURA CHIUSA DELLE OSSA MALARE E MASCELLARE SUPERIORE                            1
52669 - ALTRE PATOLOGIE PERIRADICOLARI ASSOCIATE A PRECEDENTE TRATTAMENTO ENDODONTICO      1
5286 - LEUCOPLACHIA DELLA MUCOSA ORALE,INCLUTILIZZARE LA LINGUA                            1
2101 - TUMORI BENIGNI DELLE LINGUA                                    

In [89]:
df_classified['motivo_accesso'].value_counts(normalize=True)

CARIE             0.279301
VISITA            0.237713
ASCESSO           0.210302
PARODONTOPATIE    0.175803
ALTRO             0.058601
TRAUMA            0.035917
ATM               0.002363
Name: motivo_accesso, dtype: float64

## 2. Encode Labels

In [90]:
accesso_codes = {
    'ALTRO': 0,
    'ASCESSO': 1,
    'ATM': 2,
    'CARIE': 3,
    'PARODONTOPATIE': 4,
    'TRAUMA':5
}

In [91]:
# Category mapping
df_classified['accesso_code'] = df_classified['motivo_accesso']
df_classified = df_classified.replace({'accesso_code':accesso_codes})

In [92]:
# Create file with uncategorized data
df_toclassify = df_classified[np.logical_or(df_classified['accesso_code']=='VISITA' , df_classified['accesso_code'].isnull())]

df_toclassify.to_csv('C:/Users/andrea.foroni/Downloads/visite.csv', index=False)

In [93]:
#remove visita

df_classified = df_classified[df_classified['accesso_code']!='VISITA']
df_classified = df_classified[df_classified['accesso_code'].notnull()]

In [94]:
df_classified.shape

(1613, 17)

In [105]:
df_classified['motivo_accesso'].value_counts(normalize=True)

CARIE             0.366398
ASCESSO           0.275883
PARODONTOPATIE    0.230626
ALTRO             0.076875
TRAUMA            0.047117
ATM               0.003100
Name: motivo_accesso, dtype: float64

CARIE è il motivo di accesso principale, pari al 36%. Baseline classifier che assegna sempre CARIE come motivo di accesso a PS è accurato al 36%


In [95]:
df_classified.dtypes

anno                                    int64
mese                                    int64
data_accettazione              datetime64[ns]
modalita_dimissione                    object
numero_sdo_ricetta                     object
eta_pz                                  int64
adulti-bambini                         object
data_ora_ingresso_ps           datetime64[ns]
data_ora_uscita_ps             datetime64[ns]
diagnosi_principale                    object
diagnosi_1                             object
pz_raggruppamento_residenza            object
motivo_accesso                         object
numero_pratica                         object
testo                                  object
testo_clean                            object
accesso_code                           object
dtype: object

## 3. Train - test split
We'll set apart a test set to prove the quality of our models. We'll do Cross Validation in the train set in order to tune the hyperparameters and then test performance on the unseen data of the test set.

In [96]:
X_train, X_test, y_train, y_test = train_test_split(df_classified['testo_clean'], 
                                                    df_classified['accesso_code'].astype('int64'), 
                                                    test_size=0.15, 
                                                    random_state=8)

In [101]:
# Parameter election
ngram_range = (1,2)
min_df = 0.005
max_df = 1.
max_features = 10000

In [102]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train.values.astype('U')).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test.values.astype('U')).toarray()
labels_test = y_test
print(features_test.shape)

(1371, 1344)
(242, 1344)


In [103]:
from sklearn.feature_selection import chi2

for motivo_accesso, accesso_code in sorted(accesso_codes.items()):
    features_chi2 = chi2(features_train, labels_train == accesso_code)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(motivo_accesso))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'ALTRO' category:
  . Most correlated unigrams:
. linguala
. corona
. eruzione
. fisso
. distacco
  . Most correlated bigrams:
. compressione garza
. gg rimozione

# 'ASCESSO' category:
  . Most correlated unigrams:
. ascesso
. iniezione
. ml
. drenaggio
. ceftriaxona
  . Most correlated bigrams:
. lesione radiotrasparente
. lavaggio ipoclorire

# 'ATM' category:
  . Most correlated unigrams:
. assumere
. solo
. protesi
. dolenzia
. antinfiammatoria
  . Most correlated bigrams:
. elemento 18
. pz riferire

# 'CARIE' category:
  . Most correlated unigrams:
. cariosa
. conservativa
. 75
. destruente
. carie
  . Most correlated bigrams:
. carie destruente
. evidenziare carie

# 'PARODONTOPATIE' category:
  . Most correlated unigrams:
. riassorbimento
. perdita
. parodontopatia
. supporto
. osseo
  . Most correlated bigrams:
. perdita supporto
. riassorbimento osseo

# 'TRAUMA' category:
  . Most correlated unigrams:
. ricostruzione
. frammento
. coronala
. trauma
. frattura
  . Most cor

In [104]:
# X_train
with open('results/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('results/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('results/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('results/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('results/df.pickle', 'wb') as output:
    pickle.dump(df_classified, output)
    
# features_train
with open('results/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('results/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('results/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('results/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('results/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)